In [ ]:
from snowflake.snowpark.context import get_active_session
from datetime import date
session = get_active_session()

In [ ]:
py_db = session.get_current_database()
py_raw_sch = session.get_current_schema()
py_int_sch = 'DEV_INTEGRATIONS' if 'DEV' in session.get_current_schema() else 'PROD_INTEGRATIONS'
py_har_sch = 'DEV_FRED_HARMONIZED' if 'DEV' in session.get_current_schema() else 'PROD_FRED_HARMONIZED'
py_ana_sch = 'DEV_FRED_ANALYTICS' if 'DEV' in session.get_current_schema() else 'PROD_FRED_ANALYTICS'
py_tgt = '{}.{}.FREDDATA'.format(py_db, py_raw_sch)
py_src = '@{}.{}.FRED_RAW_STAGE'.format(py_db, py_int_sch)
py_sp_tgt = '{}.{}.FREDDATA'.format(py_db.strip('"'), py_har_sch)
py_sp_src = '{}.{}.FREDDATA_STREAM'.format(py_db.strip('"'), py_raw_sch.strip('"'))
py_ff = '{}.{}.CSV_FILE_FORMAT'.format(py_db, py_int_sch)
py_mv = '{}.{}.FREDDATA'.format(py_db, py_ana_sch)
py_sp = "{}.{}.FRED_DATA_UPDATE_SP('{}', '{}')".format(py_db.strip('"'),py_int_sch, py_sp_tgt, py_sp_src)

### URL Construction

In [ ]:
py_today = date.today()
py_fred_api_base_url = 'https://api.stlouisfed.org/fred/series/observations?series_id=T10Y2Y&api_key=cf4bf1054e57bf267cb2fc89aa58b117&file_type=json&observation_start={}&observation_end={}'
py_row_count = session.sql("select count(*) as ROW_COUNT from {}".format(py_mv)).collect()[0]['ROW_COUNT']
py_fred_api_url = py_fred_api_base_url.format('2020-02-23', py_today) if py_row_count==0 else py_fred_api_base_url.format(py_today, py_today)
py_func = "{}.{}.FREDAPI_RESPONSE_TO_DF('{}')".format(py_db, py_int_sch, py_fred_api_url)

### Staging
- Source : FRED API
- Target : S3 EXTERNAL STAGE

In [ ]:
SELECT {{py_func}};

Data Ingestion
- Source : S3 EXTERNAL STAGE
- Target : FRED_DB.<ENV>_RAW_FRED.FREDDATA

In [ ]:
-- while deploying to prod, make sure to remove the force parameter
COPY INTO {{py_tgt}}
FROM {{py_src}}
FILE_FORMAT = (FORMAT_NAME = {{py_ff}})
PATTERN = '.*fred/data.csv'
FORCE = TRUE;

### Execute SP to write inserts from STREAM table into RAW SCHEMA -> FREDDATA

In [ ]:
CALL {{py_sp}};